<a href="https://colab.research.google.com/github/aayushrai/Deep-Learning/blob/master/RNN/Next_char_prediction(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load data and preprocess

In [0]:
#imports
import numpy as np

In [0]:
data = open("book.txt.txt","r").read()
chars = list(set(data))

In [0]:
 data_size ,vocab_size = len(data),len(chars)

In [0]:
char_to_int = {char:i for i,char in enumerate(chars)}
int_to_char = {i:char for i,char in enumerate(chars)}

In [0]:
print(char_to_int)

{'6': 0, 'w': 1, '!': 2, 'q': 3, 'm': 4, 't': 5, "'": 6, 'X': 7, 'c': 8, 'B': 9, '8': 10, ' ': 11, ':': 12, ';': 13, '2': 14, '/': 15, 'd': 16, 'L': 17, '-': 18, 'l': 19, 'O': 20, 'z': 21, 'D': 22, '(': 23, 'p': 24, 'V': 25, 'v': 26, '9': 27, 'N': 28, 'A': 29, 'f': 30, 'Y': 31, 'r': 32, '3': 33, '$': 34, 'j': 35, 'H': 36, 'g': 37, 'T': 38, 'F': 39, 'P': 40, '4': 41, 'U': 42, 'M': 43, 'S': 44, '0': 45, '1': 46, '5': 47, 'i': 48, ',': 49, '@': 50, 'n': 51, 'k': 52, 'a': 53, 's': 54, '%': 55, 'C': 56, 'W': 57, '?': 58, 'o': 59, '"': 60, 'J': 61, ')': 62, 'E': 63, 'y': 64, 'I': 65, 'ç': 66, '7': 67, 'e': 68, 'u': 69, 'x': 70, '.': 71, 'G': 72, 'b': 73, '\n': 74, '*': 75, 'Q': 76, 'R': 77, 'h': 78, 'K': 79}


In [0]:
print(int_to_char)

{0: '6', 1: 'w', 2: '!', 3: 'q', 4: 'm', 5: 't', 6: "'", 7: 'X', 8: 'c', 9: 'B', 10: '8', 11: ' ', 12: ':', 13: ';', 14: '2', 15: '/', 16: 'd', 17: 'L', 18: '-', 19: 'l', 20: 'O', 21: 'z', 22: 'D', 23: '(', 24: 'p', 25: 'V', 26: 'v', 27: '9', 28: 'N', 29: 'A', 30: 'f', 31: 'Y', 32: 'r', 33: '3', 34: '$', 35: 'j', 36: 'H', 37: 'g', 38: 'T', 39: 'F', 40: 'P', 41: '4', 42: 'U', 43: 'M', 44: 'S', 45: '0', 46: '1', 47: '5', 48: 'i', 49: ',', 50: '@', 51: 'n', 52: 'k', 53: 'a', 54: 's', 55: '%', 56: 'C', 57: 'W', 58: '?', 59: 'o', 60: '"', 61: 'J', 62: ')', 63: 'E', 64: 'y', 65: 'I', 66: 'ç', 67: '7', 68: 'e', 69: 'u', 70: 'x', 71: '.', 72: 'G', 73: 'b', 74: '\n', 75: '*', 76: 'Q', 77: 'R', 78: 'h', 79: 'K'}


In [0]:
#model parameters

hidden_size = 100
seq_length = 25
learning_rate = 1e-1

wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [0]:

def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # we could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input) 
    print( xs[t][inputs[t]])                                                                                                                    
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(wxh, xs[t]) + np.dot(whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dwxh, dwhh, dwhy = np.zeros_like(wxh), np.zeros_like(whh), np.zeros_like(why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #when we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dwhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dwxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dwhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(whh.T, dhraw)                                                                                                              
  return loss, dwxh, dwhh, dwhy, dbh, dby, hs[len(inputs)-1]




In [0]:
hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
p = 0 # go from start of data                                                                                                                                                             
inputs = [char_to_int[ch] for ch in data[p:p+seq_length]]
targets = [char_to_int[ch] for ch in data[p+1:p+seq_length+1]]
loss, dwxh, dwhh, dwhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
{0: array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],


In [0]:
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(wxh, x) + np.dot(whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(int_to_char[ix] for ix in ixes)
  print('----\n %s \n----' % (txt, ))

In [0]:
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_int['a'],200)

----
 nkly morn a finent roowIs. We calree t'm to hazy what conded egronk so come and he hurdain led!" azarled all reacan inst, and to spliitly sistilissed. " unl of awmed work, hurw lit lackly a laibly, ge 
----


In [0]:
p=0  
inputs = [char_to_int[ch] for ch in data[p:p+seq_length]]
print("inputs", inputs)
targets = [char_to_int[ch] for ch in data[p+1:p+seq_length+1]]
print("targets", targets)

inputs [20, 51, 68, 11, 4, 59, 32, 51, 48, 51, 37, 49, 11, 1, 78, 68, 51, 11, 72, 32, 68, 37, 59, 32, 11]
targets [51, 68, 11, 4, 59, 32, 51, 48, 51, 37, 49, 11, 1, 78, 68, 51, 11, 72, 32, 68, 37, 59, 32, 11, 44]


In [0]:
n, p = 0, 0
mwxh, mwhh, mwhy = np.zeros_like(wxh), np.zeros_like(whh), np.zeros_like(why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*500:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_int[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_int[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dwxh, dwhh, dwhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([wxh, whh, why, bh, by],
                                [dwxh, dwhh, dwhy, dbh, dby],
                                [mwxh, mwhh, mwhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                        
  n += 1 # iteration counter

iter 0, loss: 109.493696
----
 ost woa an hin alsnbpas un the tue erre ar es to to of iat ber an yom antiwwouty was and hiwt comibuacot He tke wins sin soo, bees Bonce  ite wose whir batite khous was he a ser to boose as bos in bic 
----
iter 1000, loss: 85.372920
----
 . wis panag ua yohe s.ieug saitas thecen be k was limetcilato kardto is the wot pen vo loungerrlys therithes hesdultlipthed o sis why G. d tibex, duesps sic-rp thg heuked , ly Gunm lore the kithed bes 
----
iter 2000, loss: 70.503505
----
 deyfy das e uounee lole do fuevenrel dor wetes a thek bedetaneve womeour ly trramlel whe wofhetinan so he! follr thou, samreus opinkeow Gy kutor unt lenn, iveiat tuld, basutowatke he tond pas fo coor  
----
iter 3000, loss: 63.590580
----
  outoung, hen eck he ce hat it lat shap s whet thall haghimet. wofid, hiwalnptinf anret shildim!, orend tisgta them up sooresunat.  has shis cinimens ake ma iudd the bas anlinilit at Gald wid ha lorth 
----
iter 4000, loss: 60.279524
----
  , tins o

KeyboardInterrupt: ignored